<a href="https://colab.research.google.com/github/owen198/csc_domain_adaptation/blob/main/csc_oneClassSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import preprocessing
from sklearn import svm
import pandas as pd
import datetime

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
#source_tag = 'W4662FM0400'
#target_tag = 'W4662FM0507'


source_tag = 'W4662FM0507'
target_tag = 'W4662FM0400'
path = './gdrive/My Drive/data/CSC/W4/'

In [4]:
'''
source_pd = pd.concat([pd.read_csv(path+source_tag+'_202004.csv'),
                            pd.read_csv(path+source_tag+'_202005.csv'),
                            pd.read_csv(path+source_tag+'_202006.csv'),
                            pd.read_csv(path+source_tag+'_202007.csv'),
                            pd.read_csv(path+source_tag+'_202008.csv'),
                            pd.read_csv(path+source_tag+'_202009.csv')])

target_pd = pd.concat([pd.read_csv(path+target_tag+'_202009.csv'),
                       pd.read_csv(path+target_tag+'_202010.csv'),
                       pd.read_csv(path+target_tag+'_202011.csv'),
                       pd.read_csv(path+target_tag+'_202012.csv'),
                       pd.read_csv(path+target_tag+'_202101.csv'),
                       pd.read_csv(path+target_tag+'_202102.csv')])
'''

target_pd = pd.concat([pd.read_csv(path+target_tag+'_202001.csv'),
                       pd.read_csv(path+target_tag+'_202002.csv'),
                       pd.read_csv(path+target_tag+'_202003.csv'),
                       pd.read_csv(path+target_tag+'_202004.csv'),
                       pd.read_csv(path+target_tag+'_202005.csv'),
                       pd.read_csv(path+target_tag+'_202006.csv'),
                       pd.read_csv(path+target_tag+'_202007.csv')])

source_pd = pd.concat([pd.read_csv(path+source_tag+'_202009.csv'),
                       pd.read_csv(path+source_tag+'_202010.csv'),
                       pd.read_csv(path+source_tag+'_202011.csv'),
                       pd.read_csv(path+source_tag+'_202012.csv'),
                       pd.read_csv(path+source_tag+'_202101.csv'),
                       pd.read_csv(path+source_tag+'_202102.csv')])

#W4662FM0307_pd = pd.read_csv('./gdrive/My Drive/CSC/W4/W4662FM0307.csv')
#W4662FM0307_pd.head(5)

In [5]:
source_pd['datetime'] = source_pd['timestamp'].astype('int').astype("datetime64[s]")
target_pd['datetime'] = target_pd['timestamp'].astype('int').astype("datetime64[s]")

In [6]:
target_normal = target_pd[(target_pd['datetime'] > datetime.datetime(2020,1,1,0,0)) &
                          (target_pd['datetime'] < datetime.datetime(2021,3,1,0,0))]

source_normal = source_pd[(source_pd['datetime'] > datetime.datetime(2020,9,1,0,0)) & 
                        (source_pd['datetime'] < datetime.datetime(2021,1,1,0,0))]

In [7]:
drop_list = ['_id','type','scada','timestamp','device', 'datetime']

target_normal = target_normal.drop(columns=drop_list)
source_normal = source_normal.drop(columns=drop_list)

In [8]:
def normalization (normal_df):
    
    min_max_scaler = preprocessing.MinMaxScaler()
    feature_names = list(normal_df)
    
    X_raw_minmax = min_max_scaler.fit_transform(normal_df.values)
    normal_df = pd.DataFrame(X_raw_minmax, columns=feature_names)
    
    return normal_df, min_max_scaler

In [9]:

clf = svm.OneClassSVM(nu=0.01, kernel="rbf", gamma=0.01)

source_normal, source_scaler = normalization(source_normal)
source_model = clf.fit(source_normal)

target_normal, target_scaler = normalization(target_normal)
target_model = clf.fit(target_normal)

In [10]:
def scorer_(Y_pred):
    a = (Y_pred[Y_pred == -1].size)/(Y_pred.size)
    return a*100

In [ ]:
start_date = datetime.datetime(2020, 1, 1, 0, 0)
end_date = datetime.datetime(2020, 7, 1, 0, 0)
delta = datetime.timedelta(days=1)

s_by_t = []
t_by_t = []
datetime_list = []

while start_date <= end_date:

    validation_df = target_pd[(target_pd['datetime'] > start_date) & 
                              (target_pd['datetime'] < end_date)]


    if len(validation_df) > 0:
    
        validation_df = validation_df.drop(columns=drop_list)

        validation_df_sc = source_scaler.transform(validation_df)
        validation_df_sc = source_model.predict(validation_df_sc)
        s_by_t.append(scorer_(validation_df_sc))

        validation_df_sc = target_scaler.transform(validation_df)
        validation_df_sc = target_model.predict(validation_df_sc)
        t_by_t.append(scorer_(validation_df_sc))

        datetime_list.append(start_date)

        print(start_date, scorer_(validation_df_sc))

    start_date += delta

2020-01-01 00:00:00 1.026976634061885
2020-01-02 00:00:00 1.035712686734222
2020-01-03 00:00:00 1.0387101122585767
2020-01-04 00:00:00 1.0383440787241263
2020-01-05 00:00:00 1.0461061604029445


In [ ]:
font3 = {'fontname':'SimHei'}

fig, ax = plt.subplots(figsize=(10, 5))

plt.xticks(rotation=45)

index = -100

#training_region = ax.axvspan(datetime.datetime(2021, 1, 5, 0, 0), 
#                             datetime.datetime(2021, 1, 31, 0, 0), alpha=0.2, color='yellow')

ax.plot(datetime_list[index:], s_by_t[index:], label='(source model detect target score)', marker='.')
ax.plot(datetime_list[index:], t_by_t[index:], label='(target model detect target score)', marker='.')

ax.set(xlabel='date', 
       ylabel='score')

ax.set_ylim([0, 100])


ax.legend()
ax.grid()
plt.tight_layout()
fig.savefig('score.png', dpi=300)

#fig.savefig(path+tag+'-'+training_from.strftime("%Y%m%d")+'-'+training_to.strftime("%Y%m%d")+'.png', dpi=300)
plt.show()

In [ ]:

fig, ax = plt.subplots(figsize=(25, 5))

plt.xticks(rotation=45)
ax.plot(datetime_list, t_by_t, 'o-')

failed_index = 5
failed_duration = 17

failed_region = ax.axvspan(datetime_list[failed_index], datetime_list[failed_index+failed_duration], alpha=0.2, color='red')
#training_region = ax.axvspan(training_from, training_to, alpha=0.2, color='yellow')

ax.set(xlabel='date', 
       ylabel='score',
       title=source_tag)

ax.legend([failed_region, training_region],
          ['Failure Period: '+date_list[failed_index].strftime("%Y-%m-%d") + ' ~ ' + date_list[failed_index+failed_duration].strftime("%Y-%m-%d"), 
           'Training Period'])

ax.grid()

#fig.savefig(path+source_tag+'-'+training_from.strftime("%Y%m%d")+'-'+training_to.strftime("%Y%m%d")+'.png', dpi=300)
plt.show()

In [ ]:
print)

## Here below started CPD (Change Points Detection) estimation

In [ ]:
! pip install ruptures

In [ ]:
import ruptures as rptk

In [ ]:


# detection
algo = rpt.Pelt(model="rbf").fit(np.reshape(np.array(score_list) , (-1, 1)))
result = algo.predict(pen=3)
result = [x-1 for x in result]
result

In [ ]:
fig, ax = plt.subplots(figsize=(25, 5))

plt.xticks(rotation=45)
ax.plot(date_list, score_list, 'o-')

ax.set(xlabel='date', 
       ylabel='score',
       title='Change Points Detection, tag='+ tag)

for x_pos in result:
  cpd_line = plt.axvline(date_list[x_pos], 0, 100, linestyle='--', linewidth=1, color='red')

if result[0] > 0:
    result.insert(0, 0)

if result[-1] < len(date_list)-1:
    result.insert(-1, len(date_list)-1)

for reg_pos in range(len(result)-1):
    #print(reg_pos, reg_pos+1)

    if reg_pos % 2 == 0:
        print('blue regin')
        print(date_list[result[reg_pos]], date_list[result[reg_pos+1]])
        ax.axvspan(date_list[result[reg_pos]], date_list[result[reg_pos+1]], alpha=0.1, color='green')
    elif reg_pos % 2 != 0:
        print('green regin')
        ax.axvspan(date_list[result[reg_pos]], date_list[result[reg_pos+1]], alpha=0.1, color='blue')

#ax.axvspan(date_list[result[0]-1], date_list[result[1]-1], alpha=0.2, color='green')
failed_region = ax.axvspan(date_list[150], date_list[151], alpha=0.2, color='red')
training_region = ax.axvspan(training_from, training_to, alpha=0.5, color='yellow')

ax.legend([failed_region, training_region, cpd_line],
          ['Failure Date: 6/9', 'Training Period', 'Detected Change Points'])

fig.savefig(path+tag+'_cpd.png', dpi=300)
ax.grid()
plt.show()